In [1]:
# imports
import numpy as np
from MakeFakeImage import MakeFake # a custom script to generate the images
from astropy.io import fits
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def add_noise_to_image(image, read_noise):
    noise_map = np.random.normal(0, 1, size=image.shape)
    sigma_I = np.sqrt(read_noise**2 + image)
    image += noise_map*sigma_I
    return image, sigma_I

def add_less_noise_to_image(image, read_noise):
    # 10 times less variance than add_noise_to_image
    noise_map = np.random.normal(0, 1, size=image.shape)
    sigma_I = 10**(-0.5) * np.sqrt(read_noise**2 + image)
    image += noise_map*sigma_I
    return image, sigma_I

def save_numpy_as_fits(numpy_array, filename):
    hdu = fits.PrimaryHDU(numpy_array)
    hdul = fits.HDUList([hdu])
    hdul.writeto(filename, overwrite=True)

In [3]:
# this is just a copy and paste of the code used to generate images in Section 3
print('Generating reference...')

t0 = time.time()

size = 4000
log_density = 1.5
star_density = 10**log_density # stars per 100x100 pixels
n_sources = np.int(star_density * (size**2/100**2))       
phi_r = 1 # pixels - reference image PSF standard deviation
sky = 100 # ADU

phi_k = 1.5
phi_i = np.sqrt(phi_k**2 + phi_r**2)


# set seed for rest of the image/source parameters
np.random.seed(42)

# positions
positions_x = np.random.uniform(0, size, (n_sources,1))
positions_y = np.random.uniform(0, size, (n_sources,1))
positions = np.hstack((positions_x, positions_y))

# fluxes
F = np.random.uniform(10**(-9), 10**(-4.5), n_sources)
fluxes = F**(-2./3.)

# Generate the noiseless reference image
ref_noiseless, F_frac = MakeFake(N=1, size=size, n_sources=n_sources,
                                 psf_sigma=phi_r, sky=sky,
                                 positions=positions, fluxes=fluxes, shifts=[0, 0])

print('Reference properties')
print('Reference size:', size)
print('Number of sources:', n_sources)
print('PSF standard deviation:', phi_r)
print('Sky level:', sky)
print('F_max/F_total:', F_frac) # flux ratio of brightest star that of all stars

print('\nGenerating Target Image...')
print('Kernel std:', phi_k)
print('Target Image std:', phi_i)


# Generate the noiseless (and shifted) target image
# positions
shift_x = np.random.uniform(-0.5, 0.5, 1)
shift_y = np.random.uniform(-0.5, 0.5, 1)
print('shift_x, shift_y:', shift_x, shift_y)
positions_x_shifted = positions_x + shift_x
positions_y_shifted = positions_y + shift_y
positions_shifted = np.hstack((positions_x_shifted, positions_y_shifted))

imag_noiseless, F_frac = MakeFake(N=1, size=size, n_sources=n_sources,
                                 psf_sigma=phi_i, sky=sky,
                                 positions=positions_shifted, fluxes=fluxes,
                                 shifts = [shift_x[0], shift_y[0]])    


imag_noiseless_copy = np.copy(imag_noiseless)
ref_noiseless_copy = np.copy(ref_noiseless)

# read noise [ADU]
sigma_0 = 5.

# add noise to the reference image
ref, sigma_ref = add_less_noise_to_image(image=ref_noiseless, read_noise=sigma_0)

# add noise to the target image i.e.
imag, sigma_imag = add_noise_to_image(image=imag_noiseless, read_noise=sigma_0) # need noise map for PSF fitting later on

print('Reference and target image shapes:')
print(ref.shape, imag.shape)

# calculate SNR of images
SNR_ref = np.sum(ref_noiseless_copy - sky) / np.sqrt(np.sum(sigma_ref**2))
SNR_imag = np.sum(imag_noiseless_copy - sky) / np.sqrt(np.sum(sigma_imag**2))
print('Reference SNR:', SNR_ref)
print('Target SNR:', SNR_imag)

print('\nFinished in %f seconds' % (time.time() - t0))

Generating reference...
Max flux: 763584.4109978118
Frac for 142x142 image: 0.005194979322152239
Reference properties
Reference size: 4000
Number of sources: 50596
PSF standard deviation: 1
Sky level: 100
F_max/F_total: 0.005194979322152239

Generating Target Image...
Kernel std: 1.5
Target Image std: 1.8027756377319946
shift_x, shift_y: [0.24012507] [-0.3274604]
Max flux: 763584.4109978118
Frac for 142x142 image: 0.005194979322152239
Reference and target image shapes:
(4000, 4000) (4000, 4000)
Reference SNR: 10031.334623530167
Target SNR: 3172.186538166308

Finished in 190022.313246 seconds


In [4]:
save_numpy_as_fits(ref, 'ref_speedtest.fits')
save_numpy_as_fits(imag, 'imag_speedtest.fits')